In [169]:
import datetime
from datetime import timedelta
from datetime import timezone
import tweepy
import pandas as pd
import re
from flair.models import TextClassifier
from flair.data import Sentence


API_KEY = "sVXiQKBJP7PnZTlihbigp70pK"
TOKEN_SECRET = "5mBqfX43Utio4rocfLt5oZiOI3WkZE7oEyuKoiQcHJpnNTKCFh"
ACCESS_TOKEN = "1506026300527546368-g8IHal1UhJwpS2q3pvF5lub3Yt70cA"
ACCESS_TOKEN_SECRET = "KaVduS3estbEIGBYEmvIvght3HZDKMpG39eKFSYvZlz24"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAIfyaQEAAAAAPlui3vZivPBWFSTuhwt%2F2ROsEdA%3DaxTAohTu5pw7exnBjKgpasnvPxjADUZuDI17Bd4dbA2QSEYK3S"


NUM_RESULTS = 10
CLEAN_OUT_HASHTAGS = False



client = tweepy.Client(bearer_token=BEARER_TOKEN)

def CleanText(text):
    if (CLEAN_OUT_HASHTAGS):
        return re.sub(" +", " ", re.sub("#[A-Za-z0-9_]+", "", re.sub("@[A-Za-z0-9_]+", "", text)))
    else:
        return re.sub(" +", " ", re.sub("@[A-Za-z0-9_]+", "", text))



def ProcessTweetsToDf(tweets):
    tweets_df = pd.DataFrame()
    users = {u["id"]: u for u in tweets.includes['users']}
    for tweet in tweets.data:
        # The column selections and this method of creating each data point is taken from Aparna's example
        hashtags = []
        
        username = ""
        desc = ""
        verified = False
        if users[tweet.author_id]:
            user = users[tweet.author_id]
            username = user.username
            desc = user.description
            verified = user.verified

        try:
            for hashtag in tweet.entities["hashtags"]:
                hashtags.append(hashtag["text"])
        except:
            pass
        tweets_df = tweets_df.append(pd.DataFrame({'user_name': username, 
                                                   'user_location': None,
                                                    'user_description': desc,
                                                    'user_verified': verified,
                                                    'date': tweet.created_at,
                                                    'text': tweet.text, 
                                                    'hashtags': [hashtags if hashtags else None],
                                                    'source': tweet.source,
                                                    'cleaned_text': CleanText(tweet.text)}))
        tweets_df = tweets_df.reset_index(drop=True)
    return tweets_df


# Duration is how far back we're checking as a timedelta
def SearchRecentTweets(query, duration=None):
    tweets = None
    if duration is not None:
        tweets = client.search_recent_tweets(query=query, start_time=(datetime.datetime.now() - duration), max_results = NUM_RESULTS, expansions='author_id', place_fields="full_name", tweet_fields=["created_at","source"], user_fields=["description", "verified", "username"])
    else:
        tweets = client.search_recent_tweets(query=query, max_results=NUM_RESULTS, expansions='author_id', place_fields="full_name", tweet_fields=["created_at","source"], user_fields=["description", "verified", "username"])
    
    if (tweets.data != None and len(tweets) != 0):
        return ProcessTweetsToDf(tweets)
    else:
        return None


def SearchTimePeriodTweets(query, start_time, end_time):
    tweets = client.search_recent_tweets(query=query, start_time=start_time, end_time=end_time, max_results=NUM_RESULTS, expansions='author_id', place_fields="full_name", tweet_fields=["created_at","source"], user_fields=["description", "verified", "username"])
    
    if (tweets.data != None and len(tweets) != 0):
        return ProcessTweetsToDf(tweets)
    else:
        return None







In [170]:
#print(SearchRecentTweets("Tesla", timedelta(days=2))[1][1])
#print(SearchRecentTweets("Tesla OR Elon Musk OR TSLA", timedelta(days=2))[1][1])
#print(SearchRecentTweets("#Tesla", timedelta(days=2))[1][1])
#print(SearchRecentTweets("#Tesla OR #ElonMusk", datetime.datetime(day=1, month=3, year=2022), datetime.datetime(day=1, month=3, year=2022))
#print(SearchTimePeriodTweets("#Tesla OR #ElonMusk", datetime.datetime(day=1, month=3, year=2022), datetime.datetime(day=1, month=3, year=2022)))
data = SearchRecentTweets("#Tesla", timedelta(days=2))

data.to_csv("./data.csv")

C:\Users\Christian Classen\AppData\Local\Temp\ipykernel_13032\2223430531.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tweets_df = tweets_df.append(pd.DataFrame({'user_name': username,


In [171]:
from flair.models import TextClassifier
from flair.data import Sentence

CSV_PATH = "./data.csv"

classifier = TextClassifier.load('en-sentiment')

df = pd.read_csv(CSV_PATH)
df["sentiment"] = ""

for i, text in enumerate(df["cleaned_text"]):
    sentence = Sentence(text)
    classifier.predict(sentence)
    sentiment = sentence.labels[0].to_dict()['confidence'] 
    if sentence.labels[0].to_dict()['value'] == 'NEGATIVE':
        sentiment *= -1
    df.at[i, 'sentiment'] = sentiment

df


2022-04-19 15:56:23,401 loading file C:\Users\Christian Classen\.flair\models\sentiment-en-mix-distillbert_4.pt


,Unnamed: 0,user_name,user_location,user_description,user_verified,date,text,hashtags,source,cleaned_text,sentiment
0,0,MarkWil41449149,NaN,Music 🎼 lovin Ape! Fighting for a fair market ...,False,2022-04-19 20:55:53+00:00,I’ll buy quite a few I love #Tesla and I like ...,NaN,Twitter for iPhone,I’ll buy quite a few I love #Tesla and I like ...,0.986958
1,1,bitswift_tech,NaN,Enriching lives through technology ™,False,2022-04-19 20:55:47+00:00,#Tesla and #Bitswift service vehicles. Thanks...,NaN,Twitter Web App,#Tesla and #Bitswift service vehicles. Thanks ...,0.990967
2,2,John_Inubook,NaN,$Doge - NFT CREATOR - live within the truth - ...,False,2022-04-19 20:55:46+00:00,RT @HODLdogeNOTdebt: #dogecoin #tesla 🤙🏻 https...,NaN,Twitter Web App,RT : #dogecoin #tesla 🤙🏻 https://t.co/Ltg1S454MM,0.975924
3,3,leonrvs,NaN,"Successful Familyman and tripple grandpa, F1 a...",False,2022-04-19 20:55:39+00:00,"Pff..... wat er schande, met andere woorden Te...",NaN,Twitter for iPhone,"Pff..... wat er schande, met andere woorden Te...",0.841696
4,4,silvers44502073,NaN,Super Bad Genesis Seed #635 Viral NFT check it...,False,2022-04-19 20:55:12+00:00,RT @StraightUpXRP: #Tesla settled with @SECGov...,NaN,Twitter for iPhone,"RT : #Tesla settled with on September 29, 2018...",-0.999977
5,5,jd_average,NaN,Tesla fan. Husband. Father. Veteran. FSD Beta ...,False,2022-04-19 20:55:04+00:00,Was told to Tweet as #FrunkPuppyFriday pic. Pl...,NaN,Twitter for Android,Was told to Tweet as #FrunkPuppyFriday pic. Pl...,-0.999958
6,6,MrSmithsTesla,NaN,"Electric vehicle driver in Colorado. Skier, ca...",False,2022-04-19 20:54:25+00:00,Wow! I just saved 60% on car insurance in abou...,NaN,Twitter for iPhone,Wow! I just saved 60% on car insurance in abou...,-0.998677
7,7,Hiro31366360,NaN,宣伝応援もしちゃうよ〜🌈Hawaiiが大好き❤\n株式投資初心者の為のLINEオープンチャッ...,False,2022-04-19 20:54:08+00:00,RT @kanayamafes2019: 全国のテスラオーナー必見‼️\n14日(土)開催の...,NaN,Twitter for Android,RT : 全国のテスラオーナー必見‼️\n14日(土)開催のスーパーサミットは\nテスラ車1...,0.944199
8,8,MsJonesey,NaN,NaN,False,2022-04-19 20:53:40+00:00,"@TMobile @Tesla T, T and T go hand in hand! Le...",NaN,Twitter for iPhone,"T, T and T go hand in hand! Let’s go! #teache...",0.999891
9,9,natmoonrah,NaN,Safemoon @EnhanceToken SES Squadron Member |\n...,False,2022-04-19 20:53:38+00:00,RT @SFMTruthSauce: @decent_deano @SafeMoonAce ...,NaN,Twitter for Android,RT : …,0.774311


In [172]:
yes = Sentence("hi")

classifier.predict(yes)
print(yes.labels[0])

Sentence: "hi" → NEGATIVE (0.5724)


In [173]:
OTHER_PATH = "./cleaned_sentiment_tweets.csv"
df2 = pd.read_csv(OTHER_PATH)
df2['sentiment'] = 0





In [174]:
#t = client.search_recent_tweets(query="Tesla OR Elon Musk OR TSLA", start_time=(datetime.datetime.now() - timedelta(days=6)), max_results = 100, expansions=['author_id', "geo.place_id"], place_fields="full_name", tweet_fields=["created_at", "source", "geo"], user_fields=["description", "verified", "username"])

In [175]:
type(t.includes)
#t.includes.keys()

#t.includes["users"][2].description
t.includes.keys()
#t.data[0].created_at
t.data[0].geo